In [1]:
from imports import *
from PreProcessor import *

_PreProcessor = PreProcessor()


c:\Users\basse\anaconda3\envs\cuda_env\lib\site-packages\tree_sitter\__init__.py:36: FutureWarning: Language.build_library is deprecated. Use the new bindings instead.
  warn("{} is deprecated. Use {} instead.".format(old, new), FutureWarning)
c:\Users\basse\anaconda3\envs\cuda_env\lib\site-packages\tree_sitter\__init__.py:36: FutureWarning: Language(path, name) is deprecated. Use Language(ptr, name) instead.
  warn("{} is deprecated. Use {} instead.".format(old, new), FutureWarning)


In [2]:
function_names_train,function_segments_train = _PreProcessor.setupDeepLearning("./dataset/teiid_dataset/train_CC",'CC','train')
descriptions_train, summaries_train = _PreProcessor.setupDeepLearning("./dataset/teiid_dataset/train_UC",'UC','train')
function_names_test,function_segments_test = _PreProcessor.setupDeepLearning("./dataset/teiid_dataset/test_CC",'CC','test')
descriptions_test, summaries_test = _PreProcessor.setupDeepLearning("./dataset/teiid_dataset/test_UC",'UC','test')


In [ ]:
_PreProcessor.setUpUnknown(function_names_train,function_segments_train,'CC')
_PreProcessor.setUpUnknown(descriptions_train,summaries_train,'UC')

_PreProcessor.setUpUnknown(function_names_test,function_segments_test,'CC')
_PreProcessor.setUpUnknown(descriptions_test,summaries_test,'UC')

In [ ]:
counter = 0
index = 0
for seg in function_segments_train:
    index += 1
    if len(seg) > 15000 :
        counter+=1
print(counter)

In [ ]:
with open('./pickles/DeepLearning/function_names_train.pkl', 'wb') as f:
       pickle.dump(function_names_train, f)
with open('./pickles/DeepLearning/function_segments_train.pkl', 'wb') as f:
        pickle.dump(function_segments_train, f)
with open('./pickles/DeepLearning/descriptions_train.pkl', 'wb') as f:
        pickle.dump(descriptions_train, f)
with open('./pickles/DeepLearning/summaries_train.pkl', 'wb') as f:
        pickle.dump(summaries_train, f)

with open('./pickles/DeepLearning/Vocab.pkl', 'wb') as f:
        pickle.dump(_PreProcessor.Vocab, f)

with open('./pickles/DeepLearning/function_names_test.pkl', 'wb') as f:
       pickle.dump(function_names_test, f)
with open('./pickles/DeepLearning/function_segments_test.pkl', 'wb') as f:
        pickle.dump(function_segments_test, f)
with open('./pickles/DeepLearning/descriptions_test.pkl', 'wb') as f:
        pickle.dump(descriptions_test, f)
with open('./pickles/DeepLearning/summaries_test.pkl', 'wb') as f:
        pickle.dump(summaries_test, f)

In [ ]:
function_names_train = np.load('./pickles/DeepLearning/function_names_train.pkl',allow_pickle=True)
function_segments_train = np.load('./pickles/DeepLearning/function_segments_train.pkl',allow_pickle=True)
descriptions_train = np.load('./pickles/DeepLearning/descriptions_train.pkl',allow_pickle=True)
summaries_train = np.load('./pickles/DeepLearning/summaries_train.pkl',allow_pickle=True)
vocab = np.load('./pickles/DeepLearning/Vocab.pkl',allow_pickle=True)
function_names_test = np.load('./pickles/DeepLearning/function_names_test.pkl',allow_pickle=True)
function_segments_test = np.load('./pickles/DeepLearning/function_segments_test.pkl',allow_pickle=True)
descriptions_test = np.load('./pickles/DeepLearning/descriptions_test.pkl',allow_pickle=True)
summaries_test = np.load('./pickles/DeepLearning/summaries_test.pkl',allow_pickle=True)

In [ ]:
CC_word2vec = _PreProcessor.word2VecProcessor(function_names_train, function_segments_train, 'CC')
UC_word2vec = _PreProcessor.word2VecProcessor(summaries_train, descriptions_train, 'UC')
# print(CC_word2vec)
word2vec_model = Word2Vec(sentences = CC_word2vec + UC_word2vec, vector_size=200, window=5, min_count=1, workers=4, epochs=10)
word2vec_model.save("./Dataset/teiid_dataset/UC_CC_WORD2VEC")

#word2vec_model.build_vocab(UC_word2vec, update=True)
#word2vec_model.train(UC_word2vec, total_examples=word2vec_model.corpus_count, epochs=word2vec_model.epochs)

# word2vec_model.save("./Dataset/teiid_dataset/UC_CC_WORD2VEC")
word2vec_model = Word2Vec.load("./Dataset/teiid_dataset/UC_CC_WORD2VEC")

word2vec_vocab = word2vec_model.wv.index_to_key

# Initialize an empty embedding matrix
embedding_matrix = np.zeros((len(word2vec_vocab) + 1, word2vec_model.vector_size))

# Fill the embedding matrix with the embeddings of each word
for i, word in enumerate(word2vec_vocab):
    embedding_vector = word2vec_model.wv[word]
    if embedding_vector is not None:
        embedding_matrix[i + 1] = embedding_vector



In [ ]:
print(len(embedding_matrix))

### Converting dataset to indices

In [ ]:
_PreProcessor.vocabToIndex(word2vec_vocab)
_PreProcessor.dataSetToIndex(function_names_train,function_segments_train)
_PreProcessor.dataSetToIndex(descriptions_train,summaries_train)

_PreProcessor.dataSetToIndex(function_names_test,function_segments_test)
_PreProcessor.dataSetToIndex(descriptions_test,summaries_test)

In [ ]:
x_train, labels = _PreProcessor.setUpLabels(function_names_train,function_segments_train,descriptions_train,summaries_train)

In [ ]:
with open('./pickles/DeepLearning/x_train.pkl', 'wb') as f:
       pickle.dump(x_train, f)
with open('./pickles/DeepLearning/labels.pkl', 'wb') as f:
        pickle.dump(labels, f)

In [ ]:
x_train = np.load('./pickles/DeepLearning/x_train.pkl',allow_pickle=True)
labels = np.load('./pickles/DeepLearning/labels.pkl',allow_pickle=True)

In [ ]:
class TracibilityLinkDataset(Dataset):

    def __init__(self, x_train, y_train):

        # # getting the max number of funtions in a file along all files
        # #getting the longest name of a function along all functions in all files
        # max_number_function = max(len(_) for _ in x_train_function_names)
        # max_function_name_len = max(len(function) for file in x_train_function_names for function in file)

        # #getting the longest segment of a function along all functions in all files
        # max_function_segment_len = max(len(function) for file in x_train_function_segments for function in file)

        # x_train_function_names_padded = [file + [[]] * (max_number_function - len(file)) for file in x_train_function_names]
        # x_train_function_names_padded = [function + [vocab_size+1] * (max_number_function - len(function)) for file in x_train_function_names_padded for function in file]
        
        # # convert it to a tensor to be used in model
        # self.tensor_x_train_function_names_padded = torch.tensor(x_train_function_names_padded)
        
        self.tensor_x_train = x_train
        self.tensor_y_train = torch.tensor(y_train)


    # len and get item are very important --> used by dataloader
    def __len__(self):
        return len(self.tensor_y_train)

    def __getitem__(self, idx):
        return self.tensor_x_train[idx], self.tensor_y_train[idx]


In [ ]:
dataset_train = TracibilityLinkDataset(x_train, labels)


In [ ]:
torch.save(dataset_train,'./dataset/teiid_dataset/DeepLearningDataset.pt')

In [ ]:
dataset_train = torch.load('./dataset/teiid_dataset/DeepLearningDataset.pt')

In [ ]:
class SelfAttention(nn.Module):
  def __init__(self, input_dim):
    super(SelfAttention, self).__init__()
    self.input_dim = input_dim
    self.query = nn.Linear(input_dim, input_dim) # [batch_size, seq_length, input_dim]
    self.key = nn.Linear(input_dim, input_dim) # [batch_size, seq_length, input_dim]
    self.value = nn.Linear(input_dim, input_dim)
    self.softmax = nn.Softmax(dim = 2)
   
  def forward(self, x): # x.shape (batch_size, seq_length, input_dim)
    queries = self.query(x)
    keys = self.key(x)
    values = self.value(x)

    score = torch.bmm(queries, keys.transpose(1, 2))/(self.input_dim**0.5)
    # print(torch.cuda.memory_reserved()/1024**3)

    attention = self.softmax(score)
    weighted = torch.bmm(attention, values)
    return weighted

In [ ]:
class DLModel(nn.Module):

  def __init__(self,embedding_matrix : np.array, embedding_dim : tuple, classes: int = 2):
      super(DLModel, self).__init__()

      # first embedding layer: input size --> vocab size, output size --> feature vector of vocan size(one hot encoding), weigths --> one hot encoding
      # matrix (vocab * vocab)
      self.embedding = nn.Embedding(num_embeddings = embedding_dim[0], embedding_dim = embedding_dim[1], _weight = torch.tensor(embedding_matrix))
      
      # self-attention layer --> num_heads = 1, query, key, value will all be the input
      self.attention_layer_UC = nn.MultiheadAttention(embed_dim=embedding_dim[1], num_heads=1, batch_first = True)
      self.attention_layer_CC = nn.MultiheadAttention(embed_dim=embedding_dim[1], num_heads=1, batch_first = True)

      pool_layer_2d = torch.nn.AvgPool2d(kernel_size = (5,5), stride = (2, 2))


      # softmax ?
      # self.linear = nn.Linear(2 * hidden_size_layer_chars, classes, bias=True)

  def forward(self, function_names, function_segments, descriptions, summaries ):

    # print(torch.cuda.memory_reserved()/1024**3)

    function_names_embeddings  = self.embedding(function_names)

    function_names_attention = self.attention_layer_CC(function_names_embeddings.float(),function_names_embeddings.float(),function_names_embeddings.float())
    # del function_names_embeddings
    # del function_names_attention

    function_segments_embeddings = self.embedding(function_segments)
    # print(function_segments_embeddings.element_size()*function_segments_embeddings.numel()/1024**3)


    function_segments_attention = self.attention_layer_CC(function_segments_embeddings.float(),function_segments_embeddings.float(),function_segments_embeddings.float())
    # del function_segments_embeddings
    # del function_segments_attention



    descriptions_embedding  = self.embedding(descriptions)
    descriptions_attention = self.attention_layer_UC(descriptions_embedding.float(),descriptions_embedding.float(),descriptions_embedding.float())
    # del descriptions_embedding
    # del descriptions_attention


    summaries_embedding = self.embedding(summaries)

    summaries_attention = self.attention_layer_UC(summaries_embedding.float(),summaries_embedding.float(),summaries_embedding.float())
    
    concatenated_name_seg = torch.cat((function_names_attention[0], function_segments_attention[0]), dim = 1)
    print("concatenated_name_seg.shape = " , concatenated_name_seg.shape)

    summaries_descrip_seg = torch.cat((descriptions_attention[0], summaries_attention[0]), dim = 1)
    print("summaries_descrip_seg.shape = " , summaries_descrip_seg.shape)

    name_seg = self.pool_layer_2d (concatenated_name_seg.unsqueeze(1))
    print("name_seg.shape = " , name_seg.shape)
    summary_desc = self.pool_layer_2d (summaries_descrip_seg.unsqueeze(1))
    print("summary_desc.shape = ",summary_desc.shape)


    
    
    # print(function_names_attention[0].shape)
    # print(function_segments_attention[0].shape)

    # print(descriptions_attention[0].shape)
    # print(summaries_attention[0].shape)

    # torch.cuda.empty_cache()


    # feature_vector = self.embedding_words(sentences) # sentences * words *253

    # feature_vector = feature_vector.to(dtype=torch.float)
    # lstm_output = self.lstm_layer_words(feature_vector) # sentences * words *200
    # #print(lstm_output[0].shape)  
    # # looping over lstm_output to remove unwanted feature vectors
    # no_of_sentences, length_of_sentence, feature_vector_size = lstm_output[0].shape
    # lstm_output_words= lstm_output[0]
    # lstm_output_words_2d=lstm_output_words.reshape(-1,lstm_output_words.shape[2])
    # #print(lstm_output_words_2d.shape)
    # embedding_char_output_4d=self.embedding_char(words)  # sentences * words * chars * one hot encoding for each char
    # #words * chars * one hot encoding for each char
    # embedding_char_output_3d=embedding_char_output_4d.reshape(-1,embedding_char_output_4d.shape[2],embedding_char_output_4d.shape[3])
    # #print(embedding_char_output_3d.shape)

    # #print(lstm_output_words_2d[0]) # words*hot encoding vector
    # embedding_char_output_3d_concatenated=np.zeros((embedding_char_output_3d.shape[0],embedding_char_output_3d.shape[1],embedding_char_output_3d.shape[2]+lstm_output_words_2d.shape[1]))

    # # for word in range(len(lstm_output_words_2d)):
    # #        for chars in range(len(embedding_char_output_3d[word])):
    # #print(lstm_output_words_2d.shape)
    # lstm_output_words_2d=lstm_output_words_2d.detach().numpy()

    # lstm_output_words_2d = np.repeat(lstm_output_words_2d, repeats=embedding_char_output_3d.shape[1], axis=0)
    # #print(lstm_output_words_2d.shape)

    # lstm_output_words_2d=lstm_output_words_2d.reshape(lstm_output_words_2d.shape[0]//embedding_char_output_3d.shape[1],embedding_char_output_3d.shape[1],lstm_output_words_2d.shape[1])
    # #print(lstm_output_words_2d.shape)

    # embedding_char_output_3d_concatenated = np.append(embedding_char_output_3d.detach().numpy(),lstm_output_words_2d,axis=2)  # Append along rows

    # embedding_char_output_3d_concatenated = torch.tensor(embedding_char_output_3d_concatenated)
    # #print(embedding_char_output_3d_concatenated.shape)

    # embedding_char_output_3d_concatenated = embedding_char_output_3d_concatenated.to(dtype=torch.float)
    # #lstm_output_intermediate = self.intermediate_lstm_layer(embedding_char_output_3d_concatenated)
    # #print(embedding_char_output_3d_concatenated.shape)
    # lstm_output_char = self.lstm_layer_char(embedding_char_output_3d_concatenated)
    # final_output = self.linear(lstm_output_char[0])  # words * 13 char * 15 classs
    
    return None

In [ ]:
from torch.nn.utils.rnn import pad_sequence
import copy



def customCollate(batch: list):
    # batch --> tuple (x,y)
    # x --> list of 4 lists
    x_batch, y_batch = zip(*batch)

    function_names, function_segments, descriptions , summaries = zip(*x_batch)
    # function_names_padded = list(copy.deepcopy(function_names))
    # function_segments_padded = list(copy.deepcopy(function_segments))

    # function_names -> batch_size * num_of_functions * len(function)
    # max_function_names_2nd_dim = max(len(file) for file in function_names)
    # function_names_padded = [file + [[]] * (max_function_names_2nd_dim - len(file)) for file in function_names]

    # for i, file in enumerate(function_names_padded): 
    #     max_function_names = max(len(function) for function in file)
    #     for j, function in enumerate(file):
    #         function_names_padded[i][j].extend([0] * (max_function_names - len(function)))
    #     function_names_padded[i] = torch.tensor(function_names_padded[i], dtype=torch.int64)

    # for i, file in enumerate(function_names_padded): 
    #         for j, function in enumerate(file):
    #             function_names_padded[i][j].extend([0] * (max_function_names_3rd_dim - len(function)))
    #             # function_names_padded[i][j] = torch.tensor(function_names_padded[i][j])\n",
    #         #function_names_padded[i] = torch.tensor(function_names_padded[i],dtype=torch.int64)
    
    
    # for i, file in enumerate(function_segments): 
    #     max_function_segments = max(len(function) for function in file)
    #     for j, function in enumerate(file):
    #         function_segments_padded[i][j].extend([0] * (max_function_segments - len(function)))
    #     function_segments_padded[i] = torch.tensor(function_segments_padded[i], dtype=torch.int64, device=device)


    function_names_padded = pad_sequence(function_names, batch_first=True, padding_value=0)
    function_segments_padded = pad_sequence(function_segments, batch_first=True, padding_value = 0)
    descriptions_padded = pad_sequence(descriptions, batch_first=True, padding_value=0)
    summaries_padded = pad_sequence(summaries, batch_first=True, padding_value = 0)

    
    return  function_names_padded, function_segments_padded, descriptions_padded, summaries_padded, y_batch

def train(model, train_dataset, batch_size=50, epochs=1, learning_rate=0.01):
    
    # (1) create the dataloader of the training set IMPORTANT (make the shuffle=True)
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size, collate_fn=customCollate)

    # (2) make the criterion cross entropy loss
    criterion = torch.nn.CrossEntropyLoss() 
    optimizer = torch.optim.Adam(model.parameters(),lr=0.01)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu") 
    # device= "cpu"
    if use_cuda:
        model = model.cuda()
        criterion = criterion.cuda()

    # (3) create the optimizer (Adam)

    for epoch_num in range(epochs):
        total_acc_train = 0
        total_loss_train = 0
        n_samples = 0
        batch_nu= 0
        for function_names_padded, function_segments_padded, descriptions_padded, summaries_padded, y_batch in train_dataloader:
                  
            function_names_padded_tensor = torch.tensor(function_names_padded, device=device)
            function_segments_padded_tensor = torch.tensor(function_segments_padded, device=device)
            descriptions_padded_tensor = torch.tensor(descriptions_padded, device=device)
            fsummaries_padded_tensor = torch.tensor(summaries_padded, device=device)
            print(len(function_segments_padded_tensor[0]))


            output = model.forward(function_names_padded_tensor, function_segments_padded_tensor, descriptions_padded_tensor, fsummaries_padded_tensor)
            batch_nu += 1
            # print(batch_nu)
            print(torch.cuda.memory_reserved()/1024**3)
              
            
            # (7) loss calculation (you need to think in this part how to calculate the loss correctly)

            #batch_loss = criterion(output.view(-1, output.size(2)), train_label.view(-1))

            # # (8) append the batch loss to the total_loss_train

            # total_loss_train += batch_loss
            
    #         # # (9) calculate the batch accuracy (just add the number of correct predictions)
    #         # _,predicted=torch.max(output,2)  #512 * 104  for every word in every sentence we choose one tag form the seventen tag 
    #         # acc=(predicted==train_label.view(-1, train_label.shape[2])).sum().item()
    #         # n_samples += train_label.size(0)*train_label.size(1)*train_label.size(2)
    #         # total_acc_train += acc
    #         # print(100*total_acc_train/n_samples)



    #         # # (11) do the backward pass
            #batch_loss.backward()
            #nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)

    #         # # (12) update the weights with your optimizer
            optimizer.step()
            # (10) zero your gradients
            optimizer.zero_grad()
            
            torch.cuda.empty_cache()

    #     # epoch loss
    #     # epoch_loss = total_loss_train / len(train_dataset)

    #     # # (13) calculate the accuracy
    #     # epoch_acc =100*total_acc_train/n_samples

    #     # print(
    #     #     f'Epochs: {epoch_num + 1} | Train Loss: {epoch_loss} \
    #     #     | Train Accuracy: {epoch_acc}\n')
        print("finished")
        break

  ##############################################################################################################

In [ ]:
model = DLModel(embedding_matrix, embedding_matrix.shape)
train(model, dataset_train)
# print(len(labels), len(x_train))